In [1]:
import pandas as pd
import os
from pathlib import Path
from datetime import datetime, timedelta
import json

In [2]:
folder = Path('eoles') / Path('inputs') / Path('xps')

date = datetime.now()
date = date.strftime("%Y%m%d")  # formatting
folder = folder / Path(str(date))

# add one day to date object
# date = date + timedelta(days=1)
# check if folder is a folder with library pathlib
folder.mkdir(parents=True, exist_ok=True)

base = {
    "no_subsidies": True,
    "subsidies_specified": False,
    "calibration": True,
    "max_iter": 100,
    "health": True,
    "discount_rate": 0.032,
    "weather_year": 2006,
    "carbon_constraint": True,
    "fix_sub_heater": False,
    "fix_sub_insulation": False,
    "list_year": [2020, 2025, 2030, 2035, 2040, 2045],
    "list_trajectory_scc": [0, 250, 350, 500, 650, 775],
    "eoles": {
        "biomass_potential_scenario": "S3",
        "aggregated_potential": True,
        "maximum_capacity_scenario": "N1"
    },
    "subsidy": {
            "proportional_uniform": None,
            "heater": {
                "policy": "subsidy_ad_valorem",
                "proportional": None,
                "cap": None
            },
            "insulation": {
                "policy": "subsidy_ad_valorem",
                "rational_behavior": False,
                "target": None,
                "proportional": None,
                "cap": None
            }
    },
    "scenario_cost_eoles": {
        "fix_capacities": {
            "uiom": 0,
            "CTES": 0
        }
    }
}

folder_base = folder / Path('base.json')
with open(folder_base, "w") as outfile:
    outfile.write(json.dumps(base, indent=4))

name_mapping = {
    'biomass_potential_scenario': 'biogas',
    'maximum_capacity_scenario': 'capacity',
    'demand_scenario': 'demand',
    'hourly_profile': 'profile',
    'weather': 'weather',
    'costs_supply': 'costs'
}

# scenario_mapping = {
#     'S3': 'S3',
#     'S2': 'biogas-',
#     'N1nuc': 'nuc-',
#     'N1ren2': 'ren-',
#     'Opt': 'ren+',
#     'Reindustrialisation': 'Reindustrialisation',
#     'Sobriete': 'Sobriete',
#     'hourly_profile_flat': 'flat'
# }

dict_supply = {
    'reference': {'reference': {}},
    'biomass_potential_scenario': {'Biogas-': 'S2'},
    'maximum_capacity_scenario': {'Nuc-': 'N1nuc',
                                  'Ren-': 'N1ren2',
                                  'Ren+': 'Opt'},
    'demand_scenario': {'Reindustrialisation': 'Reindustrialisation',
                        'Sobriete': 'Sobriete'},
    'hourly_profile': {'ProfileFlat': 'hourly_profile_flat'},
    'method_health_cost': {'HCHeating': 'heating_intensity'},
    'costs_supply': {
        'Hydrogen+' : {
                        'storage_capex': {'hydrogen': 0}
                    },
        'costsREN+' : {
                        'capex': 'overnight_capex_evolution_50p'
                    }
    },
    'weather' : {'2012': {'load_factors': 'vre_profiles_2012',
                  'lake_inflows': 'lake_2012',
                  'nb_years': 1,
                  'input_years': [2012]}
                 }
}

dict_demand = {
    'technical': {
        'Learning+': {
              "technical_progress": {
                "heater": {
                  "activated": True,
                  "start": 2019,
                  "end": 2035,
                  "value_end": -0.5
                }
              }
            }
    },
    'switch_heater': {
        'Elasticity-': {
          "scale": {
            "option": "price_elasticity",
            "target": -1
          }},
        'DHS2': {
          "district_heating": "project/input/energy/district_heating_s2.csv"
        }
    }
}

dict_policy_demand = {
    'Ban': {
            "file": "project/input/policies/current/policies_ambitious.json",
            "restriction_gas": {
                "start": 2025,
                "end": 2051,
                "value": "Natural gas",
                "policy": "restriction_energy"
        }},
    'BanRef': {
            "file": "project/input/policies/current/policies_ref.json",
            "restriction_gas": {
                "start": 2025,
                "end": 2051,
                "value": "Natural gas",
                "policy": "restriction_energy"
        }},
    'BanStandalone': {
        "file": "project/input/policies/current/policies_ambitious.json",
        "restriction_indiv_gas": {
          "start": 2025,
          "end": 2051,
          "value": ["Natural gas-Standard boiler", "Natural gas-Performance boiler"],
          "policy": "restriction_heater",
          "target": "Single-family"
        }
    },
    'BanNoPolicy': {
            "file": "project/input/policies/current/policies_calibration.json",
            "restriction_gas": {
                "start": 2025,
                "end": 2051,
                "value": "Natural gas",
                "policy": "restriction_energy"
        }},
    # 'CarbonTax': {
    #   "file": "project/input/policies/current/policies_calibration.json",
    #   "carbon_tax": {
    #     "start": 2017,
    #     "end": 2051,
    #     "tax": "project/input/policies/carbon_value.csv",
    #     "emission": "project/input/policies/current/carbon_tax_emission.csv",
    #     "recycling": True,
    #     "recycling_ini": 2000000000
    #   }
    # },
    'Ambitious':
        {
            "file": "project/input/policies/current/policies_ambitious.json"
        },
    'Reference':
        {
            "file": "project/input/policies/current/policies_ref.json"
        }
}

In [3]:
config_additional = {
    "name_config": "reference",
    "policies": {
        "file": "project/input/policies/current/policies_ambitious.json"
    },
    "greenfield": False,
    "prices_constant": False,
    "price_feedback": True,
    "biomass_potential_scenario": "S3",
    "aggregated_potential": True,
    "maximum_capacity_scenario": "N1",
    "lifetime_insulation": 5,
    "subsidies_heater": None,
    "subsidies_insulation": None,
    "optim_eoles": True,
    "carbon_budget": "carbon_budget_tight_convex",
    "district_heating_potential": None,
    "dict_configs": None,
    "cap_MWh": 1000,
    "cap_tCO2": 2000,
    "demand_scenario": "Reference",
    "method_health_cost": "epc"
}

list_configs = []
for supply, scenario in dict_supply.items():
    for s, name in zip(scenario.values(), [n for n in scenario.keys()]):
        print(name)
        for demand, policy in dict_policy_demand.items():
            new_config = config_additional.copy()
            new_config[supply] = s
            new_config['policies'] = policy
            if scenario == 'reference':
                name_config = 'reference'
            else:
                name_config = '_'.join([name, f'policy{demand}'])
            print(name_config)
            # name_config = '_'.join([new_name if name_mapping[supply] in x else x for x in new_config['name_config'].split('_')])
            # print(name_config)
            # name_config = name_config + f'_policy{demand}'
            new_config['name_config'] = name_config
            list_configs.append(new_config)
            folder_additional = folder / Path(name_config + '.json')
            with open(folder_additional, "w") as outfile:
                outfile.write(json.dumps(new_config, indent=4))

for supply, scenario in dict_demand.items():
    for s, name in zip(scenario.values(), [n for n in scenario.keys()]):
        print(name)
        for demand, policy in dict_policy_demand.items():
            new_config = config_additional.copy()
            new_config[supply] = s
            new_config['policies'] = policy
            if scenario == 'reference':
                name_config = 'reference'
            else:
                name_config = '_'.join([name, f'policy{demand}'])
            print(name_config)
            # new_name = name_mapping[supply] + scenario_mapping[s]
            # print(new_name)
            # name_config = '_'.join([new_name if name_mapping[supply] in x else x for x in new_config['name_config'].split('_')])
            # print(name_config)
            # name_config = name_config + f'_policy{demand}'
            new_config['name_config'] = name_config
            list_configs.append(new_config)
            folder_additional = folder / Path(name_config + '.json')
            with open(folder_additional, "w") as outfile:
                outfile.write(json.dumps(new_config, indent=4))


# list_configs = []
# for supply, scenario in dict_supply.items():
#     for s in scenario:
#         for demand, policy in dict_demand.items():
#             print(supply, s)
#             new_config = config_additional.copy()
#             new_config[supply] = s
#             new_config['policies'] = policy
#             new_name = name_mapping[supply] + scenario_mapping[s]
#             print(new_name)
#             name_config = '_'.join([new_name if name_mapping[supply] in x else x for x in new_config['name_config'].split('_')])
#             print(name_config)
#             name_config = name_config + f'_policy{demand}'
#             new_config['name_config'] = name_config
#             list_configs.append(new_config)
#             folder_additional = folder / Path(name_config + '.json')
#             with open(folder_additional, "w") as outfile:
#                 outfile.write(json.dumps(new_config, indent=4))

reference
reference_policyBan
reference_policyBanRef
reference_policyBanStandalone
reference_policyBanNoPolicy
reference_policyAmbitious
reference_policyReference
Biogas-
Biogas-_policyBan
Biogas-_policyBanRef
Biogas-_policyBanStandalone
Biogas-_policyBanNoPolicy
Biogas-_policyAmbitious
Biogas-_policyReference
Nuc-
Nuc-_policyBan
Nuc-_policyBanRef
Nuc-_policyBanStandalone
Nuc-_policyBanNoPolicy
Nuc-_policyAmbitious
Nuc-_policyReference
Ren-
Ren-_policyBan
Ren-_policyBanRef
Ren-_policyBanStandalone
Ren-_policyBanNoPolicy
Ren-_policyAmbitious
Ren-_policyReference
Ren+
Ren+_policyBan
Ren+_policyBanRef
Ren+_policyBanStandalone
Ren+_policyBanNoPolicy
Ren+_policyAmbitious
Ren+_policyReference
Reindustrialisation
Reindustrialisation_policyBan
Reindustrialisation_policyBanRef
Reindustrialisation_policyBanStandalone
Reindustrialisation_policyBanNoPolicy
Reindustrialisation_policyAmbitious
Reindustrialisation_policyReference
Sobriete
Sobriete_policyBan
Sobriete_policyBanRef
Sobriete_policyBanSta